In [1]:
import pandas as pd


df = pd.read_csv('gyro_data_clean.csv', parse_dates=['timestamp'])
df['ts'] = pd.to_datetime(df['timestamp'], unit='s')
df = df[df['ts'] >= '2025-08-20 10:00:00']
df.rename(columns={'gyro_x':'x', 'gyro_y':'y', 'gyro_z':'z'}, inplace=True)
total_samples = len(df)

/var/folders/3y/dd2_h5492gj2f1b01t_fksph0000gn/T/ipykernel_8049/3938075410.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('gyro_data_clean.csv', parse_dates=['timestamp'])
/var/folders/3y/dd2_h5492gj2f1b01t_fksph0000gn/T/ipykernel_8049/3938075410.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['ts'] = pd.to_datetime(df['timestamp'], unit='s')


In [2]:
from dash import Dash, dcc, html, Input, Output, State, dash_table
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.H2("Gyroscope Data Explorer"),
    html.Div([
        html.Label("Chart Type:"),
        dcc.Dropdown(
            id='chart-type',
            options=[
                {'label': 'Line Chart', 'value': 'line'},
                {'label': 'Scatter Plot', 'value': 'scatter'},
                {'label': 'Distribution (Histogram)', 'value': 'histogram'}
            ],
            value='line'
        ),
        html.Label("Variables:"),
        dcc.Dropdown(
            id='variables',
            options=[
                {'label': 'x', 'value': 'x'},
                {'label': 'y', 'value': 'y'},
                {'label': 'z', 'value': 'z'},
                {'label': 'All', 'value': 'all'}
            ],
            value='all',
            multi=False
        ),
        html.Label("Number of Samples:"),
        dcc.Input(
            id='num-samples',
            type='number',
            value=100,
            min=1,
            max=total_samples
        ),
        html.Button('Previous', id='prev-btn', n_clicks=0),
        html.Button('Next', id='next-btn', n_clicks=0),
    ], style={'columnCount': 2}),
    dcc.Graph(id='gyro-graph'),
    html.H4("Summary Statistics"),
    dash_table.DataTable(
        id='summary-table',
        columns=[
            {'name': stat, 'id': stat} for stat in ['variable','mean','std','min','max']
        ],
        data=[]
    )
])


In [ ]:
@app.callback(
    Output('gyro-graph', 'figure'),
    Output('summary-table', 'data'),
    Input('chart-type', 'value'),
    Input('variables', 'value'),
    Input('num-samples', 'value'),
    Input('prev-btn', 'n_clicks'),
    Input('next-btn', 'n_clicks'),
    State('gyro-graph', 'figure')
)
def update_graph(chart_type, var_sel, n_samples, prev_clicks, next_clicks, existing_fig):
    if not isinstance(n_samples, int) or n_samples <= 0:
        n_samples = total_samples

    offset = (next_clicks - prev_clicks) * n_samples
    start = max(0, min(offset, total_samples - n_samples))
    end = start + n_samples
    dff = df.iloc[start:end]
    
    if var_sel == 'all':
        cols = ['x','y','z']
    else:
        cols = [var_sel]
    
    if chart_type == 'line':
        fig = px.line(dff, y=cols, x='ts', title="Line Chart of Gyroscope")
    elif chart_type == 'scatter':
        fig = px.scatter(dff, y=cols, x='ts', title="Scatter Plot of Gyroscope")
    else:  # histogram
        fig = px.histogram(dff, x=cols, title="Distribution of Gyroscope", nbins=30)
    
    summary = []
    for c in cols:
        summary.append({
            'variable': c,
            'mean': round(dff[c].mean(), 3),
            'std': round(dff[c].std(), 3),
            'min': round(dff[c].min(), 3),
            'max': round(dff[c].max(), 3),
        })
    return fig, summary


In [6]:
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="tab", port=8055)

Dash app running on http://127.0.0.1:8055/


<IPython.core.display.Javascript object>